# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from functools import reduce
from lightgbm import LGBMClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

# Data Cleaning / Feature engineering

In [ ]:
train = pd.read_csv('D:/EYChallenge/data_train.csv')
test = pd.read_csv('D:/EYChallenge/data_test.csv')
train = train.drop(['Unnamed: 0'],axis=1)
test = test.drop(['Unnamed: 0'],axis=1)
######################################################################################################
tr = test.loc[test.x_exit.isnull()]

def conv(time_1):   
    sec1 = 3600*int(time_1[0:2]) + 60*int(time_1[3:5]) + int(time_1[6:8])                                                                                                                                
    return (sec1)
def distance(x1,x2,y1,y2):
    dist = np.sqrt( ((x2-x1)**2) + ((y2-y1)**2) )
######################################################################################################
#DISTANCE CENTRE

def distanceCentre(x,y):
    dx = max(0,abs(x_centre-x)-10000)
    dy = max(0,abs(y_centre-y)-30000)
    return (dx**2+dy**2)
distanceCentre = np.vectorize(distanceCentre)
x_centre = 3760901.5068
y_centre = -19238905.6133

train['distCenter_entry']=distanceCentre(train.x_entry,train.y_entry)
test['distCenter_entry']=distanceCentre(test.x_entry,test.y_entry)

train['distCenter_exit']=distanceCentre(train.x_exit,train.y_exit)
test['distCenter_exit']=distanceCentre(test.x_exit,test.y_exit)

train['Ecart_dist']=train['distCenter_exit']-train['distCenter_entry']
test['Ecart_dist']=test['distCenter_exit']-test['distCenter_entry']
######################################################################################################
#ANGLES

u = np.array([train.x_exit,train.y_exit])- np.array([train.x_entry,train.y_entry])
v = np.array([x_centre-train.x_entry,y_centre-train.y_entry])

angle_train = []
for i in range(u.shape[1]):
    scalar = (np.dot([u[0][i],u[1][i]],[v[0][i],v[1][i]]))
    u_norm = (np.linalg.norm([u[0][i],u[1][i]]))
    v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
    angle_train.append(math.degrees(math.acos(scalar/(u_norm*v_norm))))
    
u = np.array([test.x_exit,test.y_exit])-np.array([test.x_entry,test.y_entry])
v = np.array([x_centre-test.x_entry,y_centre-test.y_entry])

angle_test = []
for i in range(u.shape[1]):
    scalar = (np.dot([u[0][i],u[1][i]],[v[0][i],v[1][i]]))
    u_norm = (np.linalg.norm([u[0][i],u[1][i]]))
    v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
    angle_test.append(math.degrees(math.acos(scalar/(u_norm*v_norm))))

train['angle']=angle_train
test['angle']=angle_test

######################################################################################################

train.vmean[train.vmean<0]=np.NaN
train.vmin[train.vmin<0]=np.NaN
train.vmax[train.vmax<0]=np.NaN

test.vmean[test.vmean<0]=np.NaN
test.vmin[test.vmin<0]=np.NaN
test.vmax[test.vmax<0]=np.NaN

#incohérences
train['vmean'][(train.time_entry==train.time_exit)] = 0
train['vmin'][(train.time_entry==train.time_exit)] = 0
train['vmax'][(train.time_entry==train.time_exit)] = 0

test['vmean'][(test.time_entry==test.time_exit)] = 0
test['vmin'][(test.time_entry==test.time_exit)] = 0
test['vmax'][(test.time_entry==test.time_exit)] = 0

train['vmax'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0
train['vmin'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0
train['vmean'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0

test['vmax'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0
test['vmin'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0
test['vmean'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0

#outliers
train.vmean = train.vmean.loc[(train.vmean<90)]
train.vmax = train.vmean.loc[(train.vmax<90)]
train.vmin = train.vmean.loc[(train.vmin<90)]

######################################################################################################
train['vmax']=train['vmax'].replace( np.NaN , train.vmax.mean())
train['vmin']=train['vmin'].replace( np.NaN , train.vmin.mean())
train['vmean']=train['vmean'].replace( np.NaN , train.vmean.mean())
test['vmax']=test['vmax'].replace( np.NaN , test.vmax.mean())
test['vmin']=test['vmin'].replace( np.NaN , test.vmin.mean())
test['vmean']=test['vmean'].replace( np.NaN , test.vmean.mean())

######################################################################################################

vec_conv =  np.vectorize(conv)
train.time_entry=vec_conv(train.time_entry)
train.time_exit=vec_conv(train.time_exit)

test.time_entry=vec_conv(test.time_entry)
test.time_exit=vec_conv(test.time_exit)

def distance(x1,x2,y1,y2):
    dist = np.sqrt( ((x2-x1)**2) + ((y2-y1)**2) )
distance = np.vectorize(distance)
def ecartTemps(t1,t2):
    ecart= t2-t1
    return ecart
ecartTemps = np.vectorize(ecartTemps)
train['duree']=ecartTemps(train.time_entry,train.time_exit)
test['duree']=ecartTemps(test.time_entry,test.time_exit)

train['city_center'] = 0
train['city_center'][(train.x_exit>=3750901.5068) & (train.x_exit<=3770901.5068)&(train.y_exit>=(-19268905.6133)) & (train.y_exit<=(-19208905.6133))]=1

In [ ]:
# #Moments
# train['nuit']= 0
# train['nuit'][(train.time_entry<28800)]=1
# train['matin']= 0
# train['matin'][(train.time_entry<43200)&(train.time_entry>28800)]=1
# train['aprem']= 0
# train['aprem'][(train.time_entry>=43200)]=1

# test['nuit']= 0
# test['nuit'][(test.time_entry<28800)]=1
# test['matin']= 0
# test['matin'][(test.time_entry<43200)&(test.time_entry>28800)]=1
# test['aprem']= 0
# test['aprem'][(test.time_entry>=43200)]=1

In [ ]:
train = train.drop(['time_entry','time_exit'],axis=1)
test = test.drop(['time_entry','time_exit'],axis=1)

y =  train.groupby('hash')['city_center'].apply(list)
for i in range (y.shape[0]):
    y[i]=y[i][-1]
Y = pd.DataFrame(y.values)

train = train.drop(['trajectory_id','city_center'],axis=1)
test = test.drop(['trajectory_id'],axis=1)

# Convert trajectories on one line

In [ ]:
def transformOnLine(df):
    lesdataframes=[]

    for vincent in df.columns[1:]:
        print(vincent)
        k=df
        k = k.groupby('hash')[vincent].apply(list)

        if (vincent == 'distCenter_exit') | (vincent == 'x_exit') | (vincent == 'y_exit'):# | (vincent == 'Ecart_dist'):
            for i in range (k.shape[0]):
                k[i].reverse()
                k[i]=k[i][1:]      #x_exit y_exit
                
        elif (vincent == 'angle'):
            for i in range (k.shape[0]):
                try:
                    k[i]=k[i][-2]
                except(IndexError):
                    #k[i]=-1
                    k[i]=np.NaN
                    
        elif (vincent == 'Ecart_dist'):
            for i in range (k.shape[0]):
                k[i]=(np.cumsum(np.array(k[i][:-1]))).tolist()
                    
#                  k[i].reverse()
#                  k[i]=np.nanmean(np.array(k[i][1:]))

        elif (vincent == 'vmin') | (vincent == 'vmax') | (vincent == 'vmean'):
            for i in range (k.shape[0]):
                k[i].reverse()
                m=np.nanmean(np.array(k[i]))
#                 M=max(k[i])
#                 mi=min(k[i])
                k[i]=[m]
#                 last=k[i][0]
#                 if (last!=0):
#                     if (len(k[i])>1):
#                         k[i][0]=np.nanmean(np.array(k[i][1:]))
#                     else:
#                         k[i][0]=-1
                
        else :
            for i in range (k.shape[0]):
                k[i].reverse()


        k=pd.DataFrame(k)


        # expand df.time_entry into its own dataframe
        gouz = k[vincent].apply(pd.Series)

        # rename each variable is time_entry
        gouz = gouz.rename(columns = lambda x : vincent + str(x))
        
        if(vincent == 'distCenter_exit') | (vincent == 'distCenter_entry') | (vincent == 'Ecart_dist'):
            gouz = gouz.fillna(math.inf)
        elif(vincent!='angle'):
            gouz = gouz.fillna(0)

#         if(vincent == 'Ecart_dist'):
#             gouz = gouz.fillna(0)
#         else:
#             gouz = gouz.fillna(-1)
        
        
        
        lesdataframes.append(gouz)

    r = pd.concat(lesdataframes,axis=1)
    r = r.reset_index()
    r = r.drop(['hash'],axis=1)
    #r.columns = np.arange(len(r.columns))
    
#     sup = r.vmax0.loc[r.vmax0>=0]
#     r.vmax0.loc[r.vmax0<0] = sup.mean()
#     sup = r.vmin0.loc[r.vmin0>=0]
#     r.vmin0.loc[r.vmin0<0] = sup.mean()
#     sup = r.vmean0.loc[r.vmean0>=0]
#     r.vmean0.loc[r.vmean0<0] = sup.mean()
    
    return r

In [ ]:
X = transformOnLine(train)
X_test = transformOnLine(test)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
X = X
y = Y.astype(int)

# Find best hyperparameters (with stratified kfold cross-validation)

In [ ]:
# #GRID SEARCH
# X = X
# y = Y.astype(int)
# param_grid = {
#     'n_estimators':[190,200,250], 
#     'max_depth' : [9],
#     'learning_rate' : [0.09,0.1],
#     'num_leaves':[70,75],
#     'min_data_in_leaf' : [300],
#     'lambda_l2' : [0.6,0.7],
#     'colsample_bytree': [0.5,0.6],
#     'min_child_samples':[20]
# }

# for i in range(10):
#     kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
#     GridS = GridSearchCV(lgb.LGBMClassifier(n_jobs=-1),param_grid, cv=kf.split(X, y),verbose=3,n_jobs=-1, scoring= 'f1',iid=True)
#     GridS.fit(X,y)
#     print(GridS.best_params_)
#     print(GridS.best_estimator_)
# pickle.dump(GridS.best_estimator_, open('LGBMTunedV16', 'wb'))

# Find the best threshold for probabilities and evaluate the score of the models (with stratified kfold cross-validation)

In [ ]:
def findBestMeanEnsembling(listModel,nb_splits,shuffling):
    listTreshold = []
    listScore = []

    skf = StratifiedKFold(n_splits=nb_splits, shuffle=shuffling)    
    y = Y.astype(int)
    for train_index, test_index in skf.split(X, y):
        x_train, x_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        
        listPrediction = []
        for each in listModel:
            each.fit(x_train,y_train)
            prediction = each.predict_proba(x_test)
            prediction = pd.DataFrame(prediction) #proba
            prediction = prediction[1] #proba    
            listPrediction.append(prediction)
            
        ensemble_pred = pd.concat(listPrediction, ignore_index=True,axis=1)
        ensemble_pred['probaMean'] = ensemble_pred.mean(axis='columns',numeric_only=True)
        
        treshold = np.arange(0.42,0.52,0.001)
        listDic = {}
        for each in treshold:
            ensemble_pred['target'] = np.where(ensemble_pred['probaMean'] > each, 1,0)
            ensemble_pred['target']= ensemble_pred['target'].astype(int)
            score = f1_score(y_test.astype(int),ensemble_pred['target'])
            listDic[each]=score

        listKeyValue = []    
        listKeyValue.append(max(listDic.items(), key=lambda x:x[1]))

        listTreshold.append([item[0] for item in listKeyValue])
        listScore.append([item[1] for item in listKeyValue])

    print(np.mean(listTreshold))
    print(np.mean(listScore))

In [ ]:
def findOptimalEnsemble(listModel, ensemble,nb_splits,shuffling):
    ensembleClassifier = ensemble
    skf = StratifiedKFold(n_splits=nb_splits,shuffle=shuffling)
    y = Y.astype(int)
    listTreshold = []
    listScore = []

    for train_index, test_index in skf.split(X, y):
        x_train, x_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]

        listPrediction = []
        for each in listModel:
            each.fit(x_train,y_train)
            prediction = each.predict_proba(x_test)
            prediction = pd.DataFrame(prediction) #proba
            prediction = prediction[1] #proba    
            listPrediction.append(prediction)

        ensemble_pred = pd.concat(listPrediction, ignore_index=True,axis=1)
        
        xV_train, xV_test, yV_train, yV_test = train_test_split(ensemble_pred, y_test, test_size=0.3,stratify=y_test, random_state=1)
        ensembleClassifier.fit(xV_train,yV_train)
        ensemblePredict = ensembleClassifier.predict_proba(xV_test)
        ensemblePredict = pd.DataFrame(ensemblePredict)

        treshold =np.arange(0.3,0.5,0.001)
        listDic = {}
        for each in treshold:
            newPred = np.where(ensemblePredict[1] > each, 1,0)
            newPred = newPred.astype(int)
            score = f1_score(yV_test.astype(int),newPred)
            listDic[each]=score

        listKeyValue = []    
        listKeyValue.append(max(listDic.items(), key=lambda x:x[1]))
        listTreshold.append([item[0] for item in listKeyValue])
        listScore.append([item[1] for item in listKeyValue])

    print(np.mean(listTreshold))
    print(np.mean(listScore))

In [ ]:
#model1 = pickle.load(open('LGBMTunedV12','rb'))
model1 = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        importance_type='split', lambda_l2=0.5, learning_rate=0.08,
        max_depth=10, min_child_samples=20, min_child_weight=0.001,
        min_data_in_leaf=300, min_split_gain=0.0, n_estimators=200,
        n_jobs=-1, num_leaves=80, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)
#model2 = pickle.load(open('XGB_tuned_01','rb'))
model2 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=12, min_child_weight=0.4, missing=nan, n_estimators=230,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)
model3 = CatBoostClassifier(n_estimators=3000,task_type='GPU',eval_metric='F1',silent=True)

listModel = [model1,model3,model2]
for i in range(5):
    findBestMeanEnsembling(listModel,5,True)

# Train models individually on test data

In [ ]:
modelToTrain = CatBoostClassifier(n_estimators=3000,task_type='GPU',eval_metric='F1')
modelToTrain.fit(X,y)
pred = modelToTrain.predict_proba(X_test)
pred = pd.DataFrame(pred) #proba
pred = pred[1] #proba
prediction = pd.DataFrame()
trajectory_id = tr['trajectory_id']
trajectory_id=trajectory_id.values
prediction['id']=trajectory_id
prediction['target']=pred
prediction.to_csv('CBT3000est_proba_All9.csv',index=False)

In [ ]:
modelToTrain = model1
modelToTrain.fit(X,y)
pred = modelToTrain.predict_proba(X_test)
pred = pd.DataFrame(pred) #proba
pred = pred[1] #proba
prediction = pd.DataFrame()
trajectory_id = tr['trajectory_id']
trajectory_id=trajectory_id.values
prediction['id']=trajectory_id
prediction['target']=pred
prediction.to_csv("LGBMTunedV12_All9_proba.csv",index=False)

In [ ]:
modelToTrain = model2
modelToTrain.fit(X,y)
pred = modelToTrain.predict_proba(X_test)
pred = pd.DataFrame(pred) #proba
pred = pred[1] #proba
prediction = pd.DataFrame()
trajectory_id = tr['trajectory_id']
trajectory_id=trajectory_id.values
prediction['id']=trajectory_id
prediction['target']=pred
prediction.to_csv("XGB_tuned_01_All9_proba.csv",index=False)

# Ensembling probabilities with best threshold

In [ ]:
#Probas
A = pd.read_csv('CBT3000est_proba_All9.csv')
B = pd.read_csv('XGB_tuned_01_All9_proba.csv')
C = pd.read_csv('LGBMTunedV12_All9_proba.csv')

dfs = [A,B,C]
thresholds = [0.4366,0.4498,0.4476,0.4592,0.4406]

ensemble_pred = reduce(lambda left,right: pd.merge(left,right,on='id'), dfs)
ensemble_pred['probaMean'] = ensemble_pred.mean(axis='columns',numeric_only=True)
ensemble_pred = ensemble_pred[['id','probaMean']]
ensemble_pred['probaMean'] = np.where(ensemble_pred['probaMean'] >np.mean(thresholds), 1,0)
ensemble_pred['probaMean']= ensemble_pred['probaMean'].astype(int)
ensemble_pred['target']= ensemble_pred['probaMean']
ensemble_pred = ensemble_pred.drop(['probaMean'],axis=1)
ensemble_pred

In [ ]:
ensemble_pred.to_csv("LGBMV12_CBT3000est_XGBTuned_All9.csv",index=False)